In this notebook we apply some of the methods and code of shape_measurement.ipynb to investigate the effect of pixelization and undersampling on metacalibration. Important papers 
Huff & Mandelbaum https://arxiv.org/pdf/1702.02600.pdf
https://arxiv.org/pdf/1702.02601.pdf